In [1]:
from __future__ import division

In [1]:
import csv
import pandas as pd
import datetime
import string
import numpy as np
import helper as hp
import sq
import jp
import urllib

In [3]:
pd.set_option('display.max_columns', None)
Gb = 1000000000

In [4]:
#store = pd.HDFStore('access_log.h5')

In [5]:
def get_server_from_URL(URLs):
    result = []
    
    for u in URLs:
        if u.startswith('http://'):
            temp = u.split('http://')[1]
            # further split on '/' character to return only server 
            result.append(temp.split('/')[0])
        
        elif u.startswith('http://'):
            temp = u.split('https://')[1]
            # further split on '/' character to return only server 
            result.append(temp.split('/')[0])
        
    return result

In [6]:
def remove_safe_site(URLs, safe_list):
    result = []
    for u in URLs:
        m = False
        for s in safe_list:
            if s in u:
                m = True
                break
                
        if (m == False):
            result.append(u)
            
    result = pd.Series(result).unique()
        
    return result            

In [7]:
def get_match_site(URLs, safe_list):
    result = []
    for u in URLs:
        m = False
        for s in safe_list:
            if s in u:
                m = True                
                break
                                
        if (m == True):
            result.append(u)
            
    result = pd.Series(result).unique()
        
    return result   

In [8]:
#squid = sq.read_squid_log('access.log')

In [9]:
#squid.tail()

In [10]:
#safe_websites = 'google|microsoft|trendmicro|gstatic.com|bdpinsight.eu|.gov.hk'

In [11]:
#access = squid[squid['URL'].str.contains(safe_websites) == False]

In [12]:
#sum(squid.bytes)/Gb

In [16]:
def juniper_log_cleansing(x):
    if x.find('=') > 0:
        return x[x.find('=')+1:]
    elif x.find(':') > 0:
        return x[:x.find(':')]    
    else:
        return x

def read_syslog_juniper(filename):
    
    #if (filename)[-3:] == '.gz':
    #    f = hp.open_gz(filename)
    #elif (filename)[-6:] == 'tar.gz':
    #    f = hp.open_tar_gz(filename)
    #else:
    #    f = hp.open_txt(filename)
    
    chunksize = 20000
    
    #df =  pd.read_csv('311_100M.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
    df = pd.DataFrame()
    
    for c in pd.read_table(filename, sep=' {1,}|"', chunksize=chunksize, iterator=True, engine='python', index_col=False, header=None, parse_dates=[[0, 1, 2], [9,10]]):
            df = pd.concat([df, c])
            print(str(chunk) + " processed.")
    
    # restructure the dataframe
    i = [0]
    i.extend(range(3, 4))
    i.extend([1])
    i.extend(range(9, 13))
    i.extend([14])
    i.extend([16])
    i.extend(range(17, 25))
    df = df.iloc[:, i]
    
    df.columns = ['time', 'device_name', 'traffic_start', 'duration', 'policy_no', 'service', 'protocol', 'src_zone', 'dst_zone', 'action', 'sent_size', 'received_size', 'src_address', 'dst_address', 'src_port', 'dst_port', 'remarks']
    df[['device_name', 'duration', 'policy_no', 'service', 'protocol', 'src_zone', 'dst_zone', 'action', 'sent_size', 'received_size', 'src_address', 'dst_address', 'src_port', 'dst_port']] = df[['device_name', 'duration', 'policy_no', 'service', 'protocol', 'src_zone', 'dst_zone', 'action', 'sent_size', 'received_size', 'src_address', 'dst_address', 'src_port', 'dst_port']].applymap(juniper_log_cleansing)
    
    return df

In [16]:
infile = "C:\Temp\hk-ssg140.log-20150922"

In [ ]:
juniper = jp.read_syslog_juniper(infile)

In [15]:
juniper.head()

,time,device_name,traffic_start,duration,policy_no,service,protocol,src_zone,dst_zone,action,sent_size,received_size,src_address,dst_address,src_port,dst_port,remarks
0,2015-09-20 03:47:04,hk-SSG140,2015-09-20 03:59:04,0,6,tcp/port:2539,6,HK-LAN,Untrust,Deny,0,28,172.20.78.90,192.168.137.42,139,2539,session_id=0
1,2015-09-20 03:47:05,hk-SSG140,2015-09-20 03:58:05,61,397,udp/port:37738,17,AP-WAN,HK-DMZ,Permit,93,0,10.20.47.254,172.24.66.31,161,37738,src-xlated
2,2015-09-20 03:47:05,hk-SSG140,2015-09-20 03:58:04,62,587,snmp,17,HK-DMZ,HK-DMZ,Permit,90,0,172.24.66.31,172.20.68.253,47176,161,src-xlated
3,2015-09-20 03:47:05,hk-SSG140,2015-09-20 03:58:05,61,505,snmp,17,HK-DMZ,HK-LAN,Permit,92,0,172.24.66.31,172.20.78.2,42377,161,src-xlated
4,2015-09-20 03:47:05,hk-SSG140,2015-09-20 03:58:05,61,505,snmp,17,HK-DMZ,HK-LAN,Permit,92,0,172.24.66.31,172.20.78.2,37738,161,src-xlated
